# Imports

In [1]:
import webdriver_manager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import clear_output
import time
import tqdm
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import os
import json
import vlc
import bs4
import pandas as pd
import re
import numpy as np

# Getting Ready

In [6]:
#Open ChromeDriver
package = ChromeDriverManager().install()
driver = webdriver.Chrome(package)



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\harik\.wdm\drivers\chromedriver\win32\103.0.5060.53]
<ipython-input-6-b3268cc3813b>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(package)


In [2]:
books = pd.read_csv("Books_Final_with_info.csv")
books.head()

,Book Name,Author,ISBN 13,ISBN,Number of Pages,Average Rating,Total Number of Ratings,Publication Date,Date Started,Date Completed,Name,Series,Volume,Goodreads Url,Description,Genre,Image Url
0,The Lightning Tree,Patrick Rothfuss,NaN,NaN,93,4.00,2156,2014-01-01,2022-04-16,2022-04-16,The Lightning Tree,Standalone,1,https://www.goodreads.com/book/show/55262194,The Lightning Tree is a companion short story ...,"['Fantasy', 'Fiction', 'Short Stories', 'Fanta...",https://i.gr-assets.com/images/S/compressed.ph...
1,What If? Serious Scientific Answers to Absurd ...,Randall Munroe,9.780544e+12,0544272994,303,4.14,151303,2014-09-02,NaN,NaN,What If? Serious Scientific Answers to Absurd ...,What If?,1,https://www.goodreads.com/book/show/21413662,Randall Munroe left NASA in 2005 to start up h...,"['Nonfiction', 'Science', 'Humor', 'Audiobook'...",https://i.gr-assets.com/images/S/compressed.ph...
2,"Leviathan Falls (The Expanse, #9)",James S.A. Corey,9.780357e+12,0356510395,528,4.54,31066,2021-11-30,NaN,NaN,Leviathan Falls,The Expanse,9,https://www.goodreads.com/book/show/58783427,"The Laconian Empire has fallen, setting the th...","['Science Fiction', 'Fiction', 'Space', 'Space...",NaN
3,"Tiamat's Wrath (The Expanse, #8)",James S.A. Corey,9.780316e+12,0316332879,534,4.57,47842,2019-03-26,NaN,NaN,Tiamat's Wrath,The Expanse,8,https://www.goodreads.com/book/show/28335698,Thirteen hundred gates have opened to solar sy...,"['Science Fiction', 'Fiction', 'Space', 'Space...",https://i.gr-assets.com/images/S/compressed.ph...
4,"Persepolis Rising (The Expanse, #7)",James S.A. Corey,9.780316e+12,0316332828,608,4.37,53386,2017-12-05,2022-06-02,NaN,Persepolis Rising,The Expanse,7,https://www.goodreads.com/book/show/28335696,In the thousand-sun network of humanity's expa...,"['Science Fiction', 'Fiction', 'Space', 'Space...",https://i.gr-assets.com/images/S/compressed.ph...


# User Shelves List

In [1]:
def user_books_from_goodreads(user_code = None, shelf_name = None, verbose = 1):
    """
    Scarping book info from Goodreads Users's page
    the user_code is 9 digit code found in the link of user's 'My Books' page
    If no shelf name is provided, info from 'All' shelf is returned
    verbose has usual meaning with value being integer 0 or 1, default is 1.
    eg. book_from_goodreads(user_code =110673662, shelf_name = 'currently-reading', verbose = 1)
    """

    #Scarping book info from Goodreads

    #Imports
    import requests
    import bs4
    import pandas as pd

    #Containers
    books = []
    authors = []
    isbn13s = []
    isbns =[]
    pagess = []
    ratings = []
    num_ratings = []
    pub_dates = []
    date_started = []
    date_completed = []

    p=1
    while True:

        if not shelf_name:
            link = f'https://www.goodreads.com/review/list/{user_code}?page={p}'
        else:
            link = f'https://www.goodreads.com/review/list/{user_code}?page={p}&ref=nav_mybooks&shelf={shelf_name}'

        code = requests.get(link)
        soup = bs4.BeautifulSoup(code.text, "lxml")

        #Breaking the loop
        if soup.select('.bookalike') == []:
            break

        if verbose == 0:
            pass 
        else:
            print(f'Currently on Page: {p}')

        for i in range(len(soup.select('.bookalike'))):

            book_name = soup.select('.bookalike .title a')[i]['title']
            books.append(book_name)

            author = soup.select('.bookalike .author a')[i].text
            authors.append(author)

            isbn13 = soup.select('.bookalike .isbn13 .value')[i].text.strip()
            isbn13s.append(str(isbn13))
            isbn = soup.select('.bookalike .isbn .value')[i].text.strip()
            isbns.append(str(isbn))

            pages = soup.select('.bookalike .num_pages .value')[i].text.split('\n')[1].strip()
            pagess.append(pages)

            rating = float(soup.select('.bookalike .avg_rating .value')[i].text.strip())
            ratings.append(rating)

            num = soup.select('.bookalike .num_ratings .value')[i].text.strip()
            num_rating = int(''.join(num.split(',')))
            num_ratings.append(num_rating)

            pub_date = soup.select('.bookalike .date_pub .value')[i].text.strip()
            pub_dates.append(pub_date)
            
            started = soup.select('.bookalike .date_started')[i].text.split('\n')[3]
            date_started.append(started)

            ended = soup.select('.bookalike .date_read')[i].text.split('\n')[3]
            date_completed.append(ended)

        #Next page
        p=p+1

    #Creating the DataFrame
    index = 'Book Name, Author, ISBN 13, ISBN, Number of Pages, Average Rating, Total Number of Ratings, Publication Date, Date Started, Date Completed'.split(', ')
    book_df = pd.DataFrame(data = [books, authors, isbn13s, isbns, pagess, ratings, num_ratings, pub_dates, date_started, date_completed], index = index).transpose()
    return book_df

# Getting Information about the Books

## Get the Url of the Book on Goodreads

### Getting the Urls

In [21]:
book_name = "A Thousand Splendid Sun"
Author = "Khaled Hosseini"
query = "+".join(book_name.split()) + "+" + "+".join(Author.split())+"goodreads"
url = f"https://www.google.com/search?q={query}"

In [22]:
driver.get(url)
el = driver.find_element(By.CLASS_NAME , "yuRUbf")
el.find_element(By.TAG_NAME, "a").get_attribute("href")

'https://www.goodreads.com/book/show/128029.A_Thousand_Splendid_Suns'

In [58]:
def get_goodreads_link(book_name, author, series):
    if series == "Standalone":
        series = ""
    query = "+".join(book_name.split()) + "+" + "+".join(series.split())+"+" + "+".join(author.split())+ "+" + "goodreads"
    url = f"https://www.google.com/search?q={query}"
    # print(url)
    driver.get(url)
    el = driver.find_element(By.CLASS_NAME , "yuRUbf")
    book_url = el.find_element(By.TAG_NAME, "a").get_attribute("href")
    return book_url

In [59]:
authors = books['Author']
book_names = books['Name']
series = books['Series']

In [60]:
books_url = []
for i in tqdm.tqdm(range(len(books)), desc = "Getting Goodreads Links"):
    book_name = book_names[i]
    author = authors[i]
    series_name = series[i]
    try:
        book_url = get_goodreads_link(book_name, author, series_name)
        books_url.append(book_url)
    except:
        print(f"{book_name} by {author} not found")
        books_url.append(None)
    time.sleep(1)

Getting Goodreads Links: 100%|██████████| 450/450 [16:58<00:00,  2.26s/it]


### Extracting Book ID From the Urls

In [72]:
books_url[2]

'https://www.goodreads.com/book/show/58783427-leviathan-falls#:~:text=As%20nearly%20unimaginable%20forces%20prepare,and%20secrets%20if%20they%20win.'

In [75]:
id_reg = re.compile(r'\/(\d+)\/?')
id_reg.findall(books_url[2])

['58783427']

In [78]:
book_url_id = []
for url in books_url:
    if url is not None:
        id_reg.findall(url)
        try:
            book_url_id.append(id_reg.findall(url)[0])
        except:
            book_url_id.append(None)
    else:
        book_url_id.append(None)

In [80]:
book_url_id[0]

'55262194'

### Creating Final Urls

In [85]:
prefix = "https://www.goodreads.com/book/show/"
urls_final =[]
for id in book_url_id:
    if id is not None:
        urls_final.append(prefix+id)
    else:
        urls_final.append(None)
urls_final[1:5]

['https://www.goodreads.com/book/show/21413662',
 'https://www.goodreads.com/book/show/58783427',
 'https://www.goodreads.com/book/show/28335698',
 'https://www.goodreads.com/book/show/28335696']

### Appending Them To The DataFrame

In [87]:
books["Goodreads Url"] = urls_final
books.head()

,Book Name,Author,ISBN 13,ISBN,Number of Pages,Average Rating,Total Number of Ratings,Publication Date,Date Started,Date Completed,Name,Series,Volume,Goodreads Url
0,The Lightning Tree,Patrick Rothfuss,NaN,NaN,93,4.00,2156,2014-01-01,2022-04-16,2022-04-16,The Lightning Tree,Standalone,1,https://www.goodreads.com/book/show/55262194
1,What If? Serious Scientific Answers to Absurd ...,Randall Munroe,9.780544e+12,0544272994,303,4.14,151303,2014-09-02,NaN,NaN,What If? Serious Scientific Answers to Absurd ...,What If?,1,https://www.goodreads.com/book/show/21413662
2,"Leviathan Falls (The Expanse, #9)",James S.A. Corey,9.780357e+12,0356510395,528,4.54,31066,2021-11-30,NaN,NaN,Leviathan Falls,The Expanse,9,https://www.goodreads.com/book/show/58783427
3,"Tiamat's Wrath (The Expanse, #8)",James S.A. Corey,9.780316e+12,0316332879,534,4.57,47842,2019-03-26,NaN,NaN,Tiamat's Wrath,The Expanse,8,https://www.goodreads.com/book/show/28335698
4,"Persepolis Rising (The Expanse, #7)",James S.A. Corey,9.780316e+12,0316332828,608,4.37,53386,2017-12-05,2022-06-02,NaN,Persepolis Rising,The Expanse,7,https://www.goodreads.com/book/show/28335696


Let's save them.

In [88]:
books.to_csv("Books_Final.csv", index = False)

## Get the Descriptions of the Books

There are two seprate ways to get the description of the book.
1. Use the class `Foramatted` to get the description.
2. Use the id `description` to get the description.

We have to use one of them as sometimes the first one works and sometime the second one.

### First Method

#### Getting The Raw Text 

In [182]:
link = books["Goodreads Url"][2]
code = requests.get(link)
soup = bs4.BeautifulSoup(code.text, "lxml")
link

'https://www.goodreads.com/book/show/58783427'

In [195]:
soup.select(".Formatted")[0].select("span")

[]

In [183]:
raw_text = soup.select(".Formatted")[0].extract()
raw_text

IndexError: list index out of range

In [172]:
all_contents = soup.select(".Formatted")[0].contents
all_contents

['Humanity has colonized the solar system—Mars, the Moon, the Asteroid Belt and beyond—but the stars are still out of our reach.',
 <br/>,
 <br/>,
 'Jim Holden is XO of an ice miner making runs from the rings of Saturn to the mining stations of the Belt. When he and his crew stumble upon a derelict ship, the ',
 <i>Scopuli</i>,
 ', they find themselves in possession of a secret they never wanted. A secret that someone is willing to kill for—and kill on a scale unfathomable to Jim and his crew. War is brewing in the system unless he can find out who left the ship and why.',
 <br/>,
 <br/>,
 'Detective Miller is looking for a girl. One girl in a system of billions, but her parents have money and money talks. When the trail leads him to the ',
 <i>Scopuli</i>,
 ' and rebel sympathizer Holden, he realizes that this girl may be the key to everything.',
 <br/>,
 <br/>,
 'Holden and Miller must thread the needle between the Earth government, the Outer Planet revolutionaries, and secretive cor

In [173]:
all_contents[0]

'Humanity has colonized the solar system—Mars, the Moon, the Asteroid Belt and beyond—but the stars are still out of our reach.'

In [174]:
raw_text = [content.extract() for content in all_contents]
raw_text

['Humanity has colonized the solar system—Mars, the Moon, the Asteroid Belt and beyond—but the stars are still out of our reach.',
 <br/>,
 <i>Scopuli</i>,
 <br/>,
 'Detective Miller is looking for a girl. One girl in a system of billions, but her parents have money and money talks. When the trail leads him to the ',
 ' and rebel sympathizer Holden, he realizes that this girl may be the key to everything.',
 <br/>]

In [146]:
# soup.select("#description")[0].text.strip()

IndexError: list index out of range

### Using Selenium

In [309]:
link = books["Goodreads Url"][240]
code = requests.get(link)
soup = bs4.BeautifulSoup(code.text, "lxml")
link

'https://www.goodreads.com/book/show/13612965'

In [311]:
soup.select("#description")

[]

In [219]:
driver.get(link)

In [220]:
driver.find_element(By.ID, "description").find_element(By.TAG_NAME, "a").click()

In [221]:
raw_text = driver.find_element(By.ID, "description").text
raw_text = raw_text.replace("\n\n", "\n")
raw_text = raw_text.replace(" (less)", "")
raw_text

"The Seven Cities Rebellion has been crushed. Sha'ik is dead. One last rebel force remains, holed up in the city of Y'Ghatan and under the fanatical command of Leoman of the Flails. The prospect of laying siege to this ancient fortress makes the battle-weary Malaz 14th Army uneasy. For it was here that the Empire's greatest champion Dassem Ultor was slain and a tide of Malazan blood spilled. A place of foreboding, its smell is of death. But elsewhere, agents of a far greater conflict have made their opening moves. The Crippled God has been granted a place in the pantheon, a schism threatens and sides must be chosen. Whatever each god decides, the ground-rules have changed, irrevocably, terrifyingly and the first blood spilled will be in the mortal world. A world in which a host of characters, familiar and new, including Heboric Ghost Hands, the possessed Apsalar, Cutter, once a thief now a killer, the warrior Karsa Orlong and the two ancient wanderers Icarium and Mappo, each searching 

In [3]:
def get_description(driver):
    try:
        driver.find_element(By.ID, "description").find_element(By.TAG_NAME, "a").click()
    except BaseException:
        pass
    try:
        raw_text = driver.find_element(By.ID, "description").text
        raw_text = raw_text.replace("\n\n", "\n")
        raw_text = raw_text.replace(" (less)", "")
        return raw_text
    except BaseException as e:
        print("No description")
        raise e

### The Genre

In [4]:
def get_genre(driver):
    genre_list = []
    for i in range(10):
        try:
            genre = driver.find_elements(By.CLASS_NAME, "bookPageGenreLink")[i].text
            if "users" in genre:
                continue
            else:
                genre_list.append(genre)
        except BaseException:
            break
    return genre_list

In [264]:
link = books["Goodreads Url"][220]
driver.get(link)
get_genre(driver)

[]

### Get Image Url

In [265]:
driver.find_element(By.ID, "coverImage").get_attribute("src")

'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1480617604l/33220026._SY475_.jpg'

### Putting It Together

In [7]:
def get_info(url, driver=driver):
    driver.get(url)
    try:
        description = get_description(driver)
    except BaseException:
        print("No description for", url)
        description = None
    try:
        genre = get_genre(driver)
    except BaseException:
        print("No genre for", url)
        genre = None
    try:
        img_url = driver.find_element(By.ID, "coverImage").get_attribute("src")
    except BaseException:
        print("No image for", url)
        img_url = None
    return description, genre, img_url

In [357]:
#Open ChromeDriver
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\harik\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
<ipython-input-357-1d3a0ba65aca>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [8]:
driver.close()

In [345]:
link = books["Goodreads Url"][0]
get_info(url=link, driver=driver)

('The Lightning Tree is a companion short story in The Kingkiller Chronicle series. It was first published on 2014 in the anthology Rogues.',
 ['', '', '', '', '', '', '', '', '', ''],
 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1599506512l/55262194.jpg')

In [9]:
descriptions = books["Description"]
genres = books["Genre"]
img_urls = books["Image Url"]

In [53]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(package, chrome_options=chrome_options)
driver.get(books["Goodreads Url"][0])

<ipython-input-53-4e86191230b7>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(package, chrome_options=chrome_options)
<ipython-input-53-4e86191230b7>:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(package, chrome_options=chrome_options)


In [55]:
driver.get(books["Goodreads Url"][1])

In [58]:
start = 444
end = 450
i=start
for url in tqdm.tqdm(books["Goodreads Url"][start:end], desc = "Getting Info..."):
    tic=time.time()
    if url is not None:
        description, genre, img_url = get_info(url, driver=driver)
        descriptions[i] = description
        genres[i] = genre
        img_urls[i] = img_url
    else:
        print("None URL")
        time.sleep(1.5)
        pass
    toc = time.time()
    if toc-tic < 1.5:
        print("Something is Wrong. Restart the Driver")
        start = i
        break
    time.sleep(3)
    i+=1

Getting Info...:   0%|          | 0/6 [00:00<?, ?it/s]<ipython-input-58-1c43f1b8a29f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  descriptions[i] = description
<ipython-input-58-1c43f1b8a29f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genres[i] = genre
<ipython-input-58-1c43f1b8a29f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  img_urls[i] = img_url
Getting Info...: 100%|██████████| 6/6 [01:17<00:00, 12.91s/it]


In [51]:
driver.close()
i

442

In [49]:
i

431

In [41]:
books["Goodreads Url"][340]

'https://www.goodreads.com/book/show/30145666'

In [461]:
np.nonzero(descriptions)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136], dtype=int64),)

In [43]:
descriptions[339]

"Sent to a boarding school in Ancelstierre as a young child, Sabriel has had little experience with the random power of Free Magic or the Dead who refuse to stay dead in the Old Kingdom. But during her final semester, her father, the Abhorsen, goes missing, and Sabriel knows she must enter the Old Kingdom to find him.\nWith Sabriel, the first installment in the Abhorsen series, Garth Nix exploded onto the fantasy scene as a rising star, in a novel that takes readers to a world where the line between the living and the dead isn't always clear—and sometimes disappears altogether."

In [299]:
from life_saving_tools.misc.Notification import Notification

In [300]:
n = Notification()
n.send_whatsapp_text("Extraction Done!")

SMfebc074c54b043c581968d2b90314404


In [59]:
books["Description"] = descriptions
books["Genre"] = genres
books["Image Url"] = img_urls

In [60]:
books.head()

,Book Name,Author,ISBN 13,ISBN,Number of Pages,Average Rating,Total Number of Ratings,Publication Date,Date Started,Date Completed,Name,Series,Volume,Goodreads Url,Description,Genre,Image Url
0,The Lightning Tree,Patrick Rothfuss,NaN,NaN,93,4.00,2156,2014-01-01,2022-04-16,2022-04-16,The Lightning Tree,Standalone,1,https://www.goodreads.com/book/show/55262194,The Lightning Tree is a companion short story ...,"['Fantasy', 'Fiction', 'Short Stories', 'Fanta...",https://i.gr-assets.com/images/S/compressed.ph...
1,What If? Serious Scientific Answers to Absurd ...,Randall Munroe,9.780544e+12,0544272994,303,4.14,151303,2014-09-02,NaN,NaN,What If? Serious Scientific Answers to Absurd ...,What If?,1,https://www.goodreads.com/book/show/21413662,Randall Munroe left NASA in 2005 to start up h...,"['Nonfiction', 'Science', 'Humor', 'Audiobook'...",https://i.gr-assets.com/images/S/compressed.ph...
2,"Leviathan Falls (The Expanse, #9)",James S.A. Corey,9.780357e+12,0356510395,528,4.54,31066,2021-11-30,NaN,NaN,Leviathan Falls,The Expanse,9,https://www.goodreads.com/book/show/58783427,"The Laconian Empire has fallen, setting the th...","['Science Fiction', 'Fiction', 'Space', 'Space...",NaN
3,"Tiamat's Wrath (The Expanse, #8)",James S.A. Corey,9.780316e+12,0316332879,534,4.57,47842,2019-03-26,NaN,NaN,Tiamat's Wrath,The Expanse,8,https://www.goodreads.com/book/show/28335698,Thirteen hundred gates have opened to solar sy...,"['Science Fiction', 'Fiction', 'Space', 'Space...",https://i.gr-assets.com/images/S/compressed.ph...
4,"Persepolis Rising (The Expanse, #7)",James S.A. Corey,9.780316e+12,0316332828,608,4.37,53386,2017-12-05,2022-06-02,NaN,Persepolis Rising,The Expanse,7,https://www.goodreads.com/book/show/28335696,In the thousand-sun network of humanity's expa...,"['Science Fiction', 'Fiction', 'Space', 'Space...",https://i.gr-assets.com/images/S/compressed.ph...


In [61]:
books.iloc[223]

Book Name                  One, Two, Three...Infinity: Facts and Speculat...
Author                                                          George Gamow
ISBN 13                                                          9.78049e+12
ISBN                                                              0486256642
Number of Pages                                                          384
Average Rating                                                           4.2
Total Number of Ratings                                                 3498
Publication Date                                                  1947-01-01
Date Started                                                             NaN
Date Completed                                                           NaN
Name                       One, Two, Three...Infinity: Facts and Speculat...
Series                                                            Standalone
Volume                                                                     1

In [62]:
books.to_csv("Books_Final_with_info.csv", index = False)